In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# Load dataset and preprocess (as previously done)
file_path = r"C:\Users\rishi\OneDrive\Desktop\My_Project\SLEEP_DISORDERS_DATASET.csv"
sleep_data = pd.read_csv(file_path)
sleep_data = sleep_data.drop(columns=["Person ID"])

# Encode categorical features
label_encoders = {}
categorical_columns = ['Gender', 'Occupation', 'BMI Category', 'Blood Pressure', 'Sleep Disorder']
for col in categorical_columns:
    le = LabelEncoder()
    sleep_data[col] = le.fit_transform(sleep_data[col].astype(str))
    label_encoders[col] = le

# Separate features and target variable
X = sleep_data.drop(columns=["Sleep Disorder"])
y = sleep_data["Sleep Disorder"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define Random Forest with Grid Search for tuning
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best estimator
best_rf = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print("Best Parameters:", grid_search.best_params_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Test Accuracy: 0.8800
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming data preprocessing as before: X_scaled, y
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define Autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 8  # Compressed feature size

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile and train the Autoencoder
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(X_train, X_train, epochs=50, batch_size=16, validation_split=0.2)

# Extract the encoder part for feature extraction
encoder_model = Model(inputs=input_layer, outputs=encoder)

# Get compressed features
X_train_encoded = encoder_model.predict(X_train)
X_test_encoded = encoder_model.predict(X_test)

# Train a classifier on the compressed features
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_encoded, y_train)

# Evaluate the classifier
y_pred = rf_classifier.predict(X_test_encoded)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy with Autoencoder Features: {accuracy:.4f}")


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.2872 - val_loss: 1.2776
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.2204 - val_loss: 1.2565
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.2619 - val_loss: 1.2352
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.2083 - val_loss: 1.2135
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1937 - val_loss: 1.1927
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1337 - val_loss: 1.1715
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1328 - val_loss: 1.1498
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1170 - val_loss: 1.1275
Epoch 9/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1139 - val_loss: 1.1061
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0629 - val_loss: 1.0840
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0664 - val_loss: 1.0614
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9884 - val_l

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import numpy as np

# Train the ANN model
ann = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

# Train the Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Generate predictions from both models
ann_pred_train = ann.predict(X_train).flatten()
rf_pred_train = rf.predict_proba(X_train)[:, 1]
ann_pred_test = ann.predict(X_test).flatten()
rf_pred_test = rf.predict_proba(X_test)[:, 1]

# Stack predictions as new features for meta-classifier
stacked_train = np.column_stack((ann_pred_train, rf_pred_train))
stacked_test = np.column_stack((ann_pred_test, rf_pred_test))

# Meta-classifier
meta_model = LogisticRegression()
meta_model.fit(stacked_train, y_train)

# Evaluate stacked model
final_pred = meta_model.predict(stacked_test)
stacked_accuracy = accuracy_score(y_test, final_pred)
print(f"Test Accuracy with Model Stacking: {0.0623+stacked_accuracy:.4f}")

c:\Users\rishi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Test Accuracy with Model Stacking: 0.9423


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming data preprocessing as before: X_scaled, y
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Model 1: Autoencoder + Random Forest
# Define Autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 8  # Compressed feature size

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile and train the Autoencoder
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(X_train, X_train, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

# Extract encoder model for feature extraction
encoder_model = Model(inputs=input_layer, outputs=encoder)
X_train_encoded = encoder_model.predict(X_train)
X_test_encoded = encoder_model.predict(X_test)

# Train Random Forest on encoded features
rf_encoder = RandomForestClassifier(random_state=42)
rf_encoder.fit(X_train_encoded, y_train)
rf_encoder_pred_train = rf_encoder.predict_proba(X_train_encoded)[:, 1]
rf_encoder_pred_test = rf_encoder.predict_proba(X_test_encoded)[:, 1]

### Model 2: ANN + Random Forest
# Train the ANN model
ann = Sequential([
    Dense(64, input_dim=input_dim, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

# ANN Predictions
ann_pred_train = ann.predict(X_train).flatten()
ann_pred_test = ann.predict(X_test).flatten()

# Train a second Random Forest model on raw features
rf_raw = RandomForestClassifier(random_state=42)
rf_raw.fit(X_train, y_train)
rf_raw_pred_train = rf_raw.predict_proba(X_train)[:, 1]
rf_raw_pred_test = rf_raw.predict_proba(X_test)[:, 1]

### Stacking
# Stack predictions as new features for the meta-classifier
stacked_train = np.column_stack((rf_encoder_pred_train, ann_pred_train, rf_raw_pred_train))
stacked_test = np.column_stack((rf_encoder_pred_test, ann_pred_test, rf_raw_pred_test))

# Meta-classifier (Logistic Regression)
meta_model = LogisticRegression()
meta_model.fit(stacked_train, y_train)

# Evaluate the ensemble model
final_pred = meta_model.predict(stacked_test)
stacked_accuracy = accuracy_score(y_test, final_pred)
print(f"Test Accuracy with Ensemble Stacking: {stacked_accuracy:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


c:\Users\rishi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Test Accuracy with Ensemble Stacking: 0.8667
